# Activity Classification - DNN1 Training

This notebook trains our first Deep Neural Network on the physical activity dataset.


## Load Data and Prepare Training Set

In [ ]:
%reset -f

import importlib
import sys, os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import activity_functions
importlib.reload(activity_functions)
from activity_functions import *

ModuleNotFoundError: No module named 'kagglehub'

In [ ]:
activtity = load_data()

In [ ]:
df_train, df_test = create_train_test(activtity, test_ratio=0.2)
print(df_train.shape)
print(df_test.shape)

In [ ]:
X_train, y_train, X_test, y_test = prepare_for_train(df_train, df_test)

## Model Fit

## HYPERTUNE SOMEWHERE

## Model Evaluation

## Summary

Discuss results